In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True, reshape=False)

import tensorflow as tf

# parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# number of samples to calculate validation and accuracy
# decrease this if you're running out of memory
test_valid_size = 256

# network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # dropout (probability to keep units)

weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch  1, Batch   1 -Loss: 52899.0117 Validation Accuracy: 0.140625
Epoch  1, Batch   2 -Loss: 42194.1328 Validation Accuracy: 0.218750
Epoch  1, Batch   3 -Loss: 28551.5820 Validation Accuracy: 0.234375
Epoch  1, Batch   4 -Loss: 25297.1621 Validation Accuracy: 0.222656
Epoch  1, Batch   5 -Loss: 23119.3438 Validation Accuracy: 0.214844
Epoch  1, Batch   6 -Loss: 22490.2734 Validation Accuracy: 0.214844
Epoch  1, Batch   7 -Loss: 22940.0859 Validation Accuracy: 0.207031
Epoch  1, Batch   8 -Loss: 17582.5547 Validation Accuracy: 0.214844
Epoch  1, Batch   9 -Loss: 21032.5723 Validation Accuracy: 0.253906
Epoch  1, Batch  10 -Loss: 18075.4473 Validation Accuracy: 0.246094
Epoch  1, Batch  11 -Loss: 18152.8555 Validation Accuracy: 0.261719
Epoch  1, Batch  12 -Loss: 16981.3672 Validation Accuracy:

Epoch  1, Batch 119 -Loss:  3577.3213 Validation Accuracy: 0.578125
Epoch  1, Batch 120 -Loss:  3876.7725 Validation Accuracy: 0.605469
Epoch  1, Batch 121 -Loss:  3847.7893 Validation Accuracy: 0.609375
Epoch  1, Batch 122 -Loss:  3769.7771 Validation Accuracy: 0.609375
Epoch  1, Batch 123 -Loss:  4806.7979 Validation Accuracy: 0.617188
Epoch  1, Batch 124 -Loss:  4355.6025 Validation Accuracy: 0.605469
Epoch  1, Batch 125 -Loss:  3495.6377 Validation Accuracy: 0.617188
Epoch  1, Batch 126 -Loss:  3730.5337 Validation Accuracy: 0.621094
Epoch  1, Batch 127 -Loss:  3684.4771 Validation Accuracy: 0.625000
Epoch  1, Batch 128 -Loss:  3393.7358 Validation Accuracy: 0.628906
Epoch  1, Batch 129 -Loss:  4174.2944 Validation Accuracy: 0.632812
Epoch  1, Batch 130 -Loss:  3071.8237 Validation Accuracy: 0.628906
Epoch  1, Batch 131 -Loss:  4221.5151 Validation Accuracy: 0.640625
Epoch  1, Batch 132 -Loss:  4100.8018 Validation Accuracy: 0.632812
Epoch  1, Batch 133 -Loss:  3184.0596 Validation

Epoch  1, Batch 240 -Loss:  2141.3059 Validation Accuracy: 0.726562
Epoch  1, Batch 241 -Loss:  2604.6118 Validation Accuracy: 0.718750
Epoch  1, Batch 242 -Loss:  2514.7454 Validation Accuracy: 0.714844
Epoch  1, Batch 243 -Loss:  1923.6761 Validation Accuracy: 0.714844
Epoch  1, Batch 244 -Loss:  2108.8079 Validation Accuracy: 0.718750
Epoch  1, Batch 245 -Loss:  3219.6284 Validation Accuracy: 0.710938
Epoch  1, Batch 246 -Loss:  1653.2446 Validation Accuracy: 0.714844
Epoch  1, Batch 247 -Loss:  2112.1489 Validation Accuracy: 0.710938
Epoch  1, Batch 248 -Loss:  1994.7124 Validation Accuracy: 0.703125
Epoch  1, Batch 249 -Loss:  2082.9541 Validation Accuracy: 0.703125
Epoch  1, Batch 250 -Loss:  2043.1869 Validation Accuracy: 0.703125
Epoch  1, Batch 251 -Loss:  2962.2500 Validation Accuracy: 0.707031
Epoch  1, Batch 252 -Loss:  2006.7668 Validation Accuracy: 0.718750
Epoch  1, Batch 253 -Loss:  1602.3357 Validation Accuracy: 0.714844
Epoch  1, Batch 254 -Loss:  1904.6503 Validation

Epoch  1, Batch 361 -Loss:  1290.7037 Validation Accuracy: 0.718750
Epoch  1, Batch 362 -Loss:  1348.7539 Validation Accuracy: 0.722656
Epoch  1, Batch 363 -Loss:  1454.1580 Validation Accuracy: 0.722656
Epoch  1, Batch 364 -Loss:  1965.4712 Validation Accuracy: 0.722656
Epoch  1, Batch 365 -Loss:  1841.5724 Validation Accuracy: 0.718750
Epoch  1, Batch 366 -Loss:  1627.3003 Validation Accuracy: 0.718750
Epoch  1, Batch 367 -Loss:  1602.4282 Validation Accuracy: 0.718750
Epoch  1, Batch 368 -Loss:  1579.5029 Validation Accuracy: 0.718750
Epoch  1, Batch 369 -Loss:  1705.2656 Validation Accuracy: 0.714844
Epoch  1, Batch 370 -Loss:  1632.7153 Validation Accuracy: 0.718750
Epoch  1, Batch 371 -Loss:  1899.7131 Validation Accuracy: 0.718750
Epoch  1, Batch 372 -Loss:  1740.4839 Validation Accuracy: 0.718750
Epoch  1, Batch 373 -Loss:  1155.3293 Validation Accuracy: 0.734375
Epoch  1, Batch 374 -Loss:  1600.7566 Validation Accuracy: 0.718750
Epoch  1, Batch 375 -Loss:  1817.5697 Validation

Epoch  2, Batch  53 -Loss:  1331.1074 Validation Accuracy: 0.746094
Epoch  2, Batch  54 -Loss:  1241.0947 Validation Accuracy: 0.750000
Epoch  2, Batch  55 -Loss:  1561.9661 Validation Accuracy: 0.750000
Epoch  2, Batch  56 -Loss:  1177.7534 Validation Accuracy: 0.750000
Epoch  2, Batch  57 -Loss:  1650.8756 Validation Accuracy: 0.753906
Epoch  2, Batch  58 -Loss:  1067.3763 Validation Accuracy: 0.753906
Epoch  2, Batch  59 -Loss:  1080.9314 Validation Accuracy: 0.765625
Epoch  2, Batch  60 -Loss:  1293.4336 Validation Accuracy: 0.757812
Epoch  2, Batch  61 -Loss:  1711.0935 Validation Accuracy: 0.757812
Epoch  2, Batch  62 -Loss:  1478.4803 Validation Accuracy: 0.761719
Epoch  2, Batch  63 -Loss:  1249.0908 Validation Accuracy: 0.757812
Epoch  2, Batch  64 -Loss:  1401.7058 Validation Accuracy: 0.757812
Epoch  2, Batch  65 -Loss:  1658.0461 Validation Accuracy: 0.757812
Epoch  2, Batch  66 -Loss:  1159.6750 Validation Accuracy: 0.757812
Epoch  2, Batch  67 -Loss:  1485.8744 Validation

Epoch  2, Batch 174 -Loss:  1368.3989 Validation Accuracy: 0.761719
Epoch  2, Batch 175 -Loss:  1012.4186 Validation Accuracy: 0.765625
Epoch  2, Batch 176 -Loss:  1027.2505 Validation Accuracy: 0.761719
Epoch  2, Batch 177 -Loss:  1098.5891 Validation Accuracy: 0.769531
Epoch  2, Batch 178 -Loss:  1067.9817 Validation Accuracy: 0.761719
Epoch  2, Batch 179 -Loss:  1142.3112 Validation Accuracy: 0.757812
Epoch  2, Batch 180 -Loss:   947.0784 Validation Accuracy: 0.761719
Epoch  2, Batch 181 -Loss:  1076.0952 Validation Accuracy: 0.757812
Epoch  2, Batch 182 -Loss:  1006.0078 Validation Accuracy: 0.765625
Epoch  2, Batch 183 -Loss:  1090.8499 Validation Accuracy: 0.765625
Epoch  2, Batch 184 -Loss:  1191.2789 Validation Accuracy: 0.773438
Epoch  2, Batch 185 -Loss:  1138.0182 Validation Accuracy: 0.769531
Epoch  2, Batch 186 -Loss:  1560.7036 Validation Accuracy: 0.773438
Epoch  2, Batch 187 -Loss:   991.8530 Validation Accuracy: 0.765625
Epoch  2, Batch 188 -Loss:  1143.1450 Validation

Epoch  2, Batch 295 -Loss:  1082.8071 Validation Accuracy: 0.765625
Epoch  2, Batch 296 -Loss:  1152.8311 Validation Accuracy: 0.773438
Epoch  2, Batch 297 -Loss:   796.7233 Validation Accuracy: 0.781250
Epoch  2, Batch 298 -Loss:  1151.4437 Validation Accuracy: 0.773438
Epoch  2, Batch 299 -Loss:   941.9850 Validation Accuracy: 0.773438
Epoch  2, Batch 300 -Loss:   797.5930 Validation Accuracy: 0.765625
Epoch  2, Batch 301 -Loss:  1155.9114 Validation Accuracy: 0.765625
Epoch  2, Batch 302 -Loss:  1389.3677 Validation Accuracy: 0.769531
Epoch  2, Batch 303 -Loss:   818.8186 Validation Accuracy: 0.769531
Epoch  2, Batch 304 -Loss:  1107.7886 Validation Accuracy: 0.773438
Epoch  2, Batch 305 -Loss:  1099.5190 Validation Accuracy: 0.765625
Epoch  2, Batch 306 -Loss:  1134.0659 Validation Accuracy: 0.765625
Epoch  2, Batch 307 -Loss:   547.3203 Validation Accuracy: 0.769531
Epoch  2, Batch 308 -Loss:  1038.3381 Validation Accuracy: 0.773438
Epoch  2, Batch 309 -Loss:  1163.8035 Validation

Epoch  2, Batch 416 -Loss:   817.0593 Validation Accuracy: 0.773438
Epoch  2, Batch 417 -Loss:   629.4625 Validation Accuracy: 0.781250
Epoch  2, Batch 418 -Loss:  1188.3254 Validation Accuracy: 0.781250
Epoch  2, Batch 419 -Loss:  1043.3799 Validation Accuracy: 0.777344
Epoch  2, Batch 420 -Loss:   816.0814 Validation Accuracy: 0.773438
Epoch  2, Batch 421 -Loss:  1056.8276 Validation Accuracy: 0.765625
Epoch  2, Batch 422 -Loss:   889.3193 Validation Accuracy: 0.773438
Epoch  2, Batch 423 -Loss:   988.5483 Validation Accuracy: 0.777344
Epoch  2, Batch 424 -Loss:  1097.0022 Validation Accuracy: 0.785156
Epoch  2, Batch 425 -Loss:   708.9195 Validation Accuracy: 0.785156
Epoch  2, Batch 426 -Loss:  1041.8627 Validation Accuracy: 0.781250
Epoch  2, Batch 427 -Loss:  1414.0488 Validation Accuracy: 0.777344
Epoch  2, Batch 428 -Loss:  1081.8721 Validation Accuracy: 0.777344
Epoch  2, Batch 429 -Loss:   821.8173 Validation Accuracy: 0.777344
Epoch  3, Batch   1 -Loss:  1159.8833 Validation

Epoch  3, Batch 108 -Loss:   872.5099 Validation Accuracy: 0.765625
Epoch  3, Batch 109 -Loss:   849.5314 Validation Accuracy: 0.765625
Epoch  3, Batch 110 -Loss:   851.1302 Validation Accuracy: 0.757812
Epoch  3, Batch 111 -Loss:  1363.1252 Validation Accuracy: 0.757812
Epoch  3, Batch 112 -Loss:   984.7922 Validation Accuracy: 0.757812
Epoch  3, Batch 113 -Loss:   617.9857 Validation Accuracy: 0.765625
Epoch  3, Batch 114 -Loss:   748.7897 Validation Accuracy: 0.765625
Epoch  3, Batch 115 -Loss:   946.9650 Validation Accuracy: 0.757812
Epoch  3, Batch 116 -Loss:  1014.0403 Validation Accuracy: 0.750000
Epoch  3, Batch 117 -Loss:   986.4552 Validation Accuracy: 0.765625
Epoch  3, Batch 118 -Loss:   948.4716 Validation Accuracy: 0.769531
Epoch  3, Batch 119 -Loss:   501.7422 Validation Accuracy: 0.765625
Epoch  3, Batch 120 -Loss:   804.7251 Validation Accuracy: 0.765625
Epoch  3, Batch 121 -Loss:   247.5794 Validation Accuracy: 0.769531
Epoch  3, Batch 122 -Loss:   710.3219 Validation

KeyboardInterrupt: 